# **SIMS: jupyter notebook for simple training and inference**

This notebook is designed to assist in running SIMS to perform basic training, or for cell type inference using our pre-trained model checkpoints. We recommend using this notebook within GitHub Codespaces. If you are looking for a more advanced training customization, please refer to the SIMS GitHub repository for instructions on using an API: https://github.com/braingeneers/SIMS

## Getting started

To get started, drag and drop your `.h5ad` files into the `codespaces` folder for easy access.

Locate and run the `setup_sims_env.sh` file within this directory to set up your virtual environment.

- In your terminal, execute the following commands:

    `chmod +x setup_sims_env.sh`

    `./setup_sims_env.sh` 

- This ensures that SIMS runs with the correct software dependencies. 


After executing the script, make sure to select the newly created `sims_env` environment as the kernel for this notebook. It is essential to use Python3.9.

Next, download the following libraries:

In [ ]:
from scsims import SIMS
import pandas as pd
import anndata as an

-----------------

## Training

**Note:** Use this section to train your own model using a labeled dataset. If you would just like to make predictions using a pretrained model checkpoint from our checkpoint folder, **please skip to the Inference section.**

**Note:** For custom training jobs and logging, we recommend running SIMS via an API. Please refer to our GitHub for instructions: https://github.com/braingeneers/SIMS

#### Load training data and initialize SIMS model

- Don't forget to replace the file paths and class label!

- Your training data should be located in this folder (`codespaces`), assuming you uploaded your `.h5ad` files here for easy access. 

In [ ]:
labeled_data_path = "/my_labeled_data.h5ad"  # Replace 'my_labeled_data.h5ad' with the labeled
                                             # data you would like to use for training.
                                            
train_data = an.read_h5ad(labeled_data_path)

sims = SIMS(data=train_data, class_label='class_label')  # Change 'class_label' to be the variable you are predicting for

#### Begin training

- Checkpoints will save automatically to a `lightning_logs` directory in the current working directory

In [ ]:
sims.train()  # Train the model

--------

## Inference

#### Load data and initialize SIMS model

- Don't forget to change the file paths!

- Your test data should be located in this folder (`codespaces`), assuming you uploaded your `.h5ad` files here for easy access. 

In [ ]:
checkpoint_path = '../checkpoint/myawesomemodel.ckpt'   # Replace '../checkpoint/myawesomemodel.ckpt' with the path to a
                                                        # pretrained model from the sims_app/checkpoint folder. Alternatively, 
                                                        # you can load your own .ckpt file developed during training from the 
                                                        # 'lightning_logs' directory  
                                                        
test_data_path = '/my_test_data.h5ad'  # Replace 'my_test_data.h5ad' with the data you would like
                                       # to perform predictions on using your chosen model checkpoint. 

sims = SIMS(weights_path=checkpoint_path)  # Initialize the SIMS model with the checkpoint

#### Run and save predictions

- Uses `sims.predict` to run predictions on test data with your chosen checkpoint.

- Saves predictions to a `.csv` file.

In [ ]:
cell_predictions = sims.predict(test_data_path)  # Predict

cell_predictions.to_csv('predictions.csv')  # Save

#### View the explainability of the model



- Assesses the importance of different genes in making predictions.

- Uses `sims.explain`(`test_data_path`) to create an explainability matrix for the test data.

- Obtains the list of gene names from the SIMS model using `sims.model.genes`.

- Constructs a Pandas DataFrame, `explain`, using the explainability matrix and gene names as column headers.

- Computes the mean explanation score for each gene across all samples with `explain.mean(axis=0`)`.

- Determines the variance of the explanation scores for each gene across all samples using `explain.var(axis=0)`.

- Combines the mean and variance data into a new Pandas DataFrame, `mean_and_var_data`, and labels the columns as `mean_explain` and `var_explain`.

In [ ]:
explainability_matrix = sims.explain(test_data_path)  # Generate explainability matrix
gene_names = sims.model.genes  # Retrieve the gene names from the SIMS model

explain = pd.DataFrame(explainability_matrix, columns=gene_names)   # Create a Pandas DataFrame from the explainability
                                                                    # matrix, using gene names as columns

mean_explain = explain.mean(axis=0)     # Mean explanation score for each gene across all samples
var_explain = explain.var(axis=0)   # Variance of explanation score for each gene across all samples

mean_and_var_data = pd.concat([mean_explain, var_explain], axis=1)  # Pandas dataframe containing information on gene importance
mean_and_var_data.columns = ["mean_explain", "var_explain"]